# __CNN__

We decide to use CNN(convolutional neural networks) for the task of guitar tablature estimation. The previous work of Andrew Wiggins and Youngmoo Kim showed that CNNs have shown promise for translating guitar audios to tabs, and the use of CNNs has also been explored for various other tasks within music information retrieval such as musical tempo estimation, key classification, singing voice detection, and instrument classification. It is proven that CNN is a powerful tool for the purpose of our study.

## __Import libraries__

In [17]:
# Import required packages 
import tensorflow as tf
import tensorflow_hub as hub
import datetime
import pathlib
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from keras import backend as K
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

# Load the TensorBoard notebook extension
%load_ext tensorboard

RSEED = 42

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
#Clear any logs from previous runs
!rm -rf ./logs/

In [6]:
# Check for Tensorflow version
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

2.8.0


## __"Write Python Script" function__

`%%write_and_run image_modeling.py` is the call of the register cell magic from below in 'w' mode (default). It writes the imports at the beginning of the `image_modeling.py`.

In [ ]:
'''# Let's make some dark cell magic. Why not!

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
        print("Appended to file ", file)
    else:
        print('Written to file:', file)
    with open(file, mode) as f:
        f.write(cell.format(**globals()))        
    get_ipython().run_cell(cell)'''

## __Define Input Shapes__

In [ ]:
#Import variables from our file
FRAME_HEIGHT = 192
FRAME_WIDTH = 9
N_CLASSES = 21
N_STRINGS = 6
BATCH_SIZE = 128
EPOCHS = 8

TRAIN_PATH = 'our trainset path.csv'
EVAL_PATH = 'our evalset path.csv'
TEST_PATH = 'our testset path.csv'

#TRAINING_SIZE = !wc -l < flowers_train.csv
#TRAINING_STEPS = int(TRAINING_SIZE[0]) // BATCH_SIZE




Using Tensorboard to monitor our results:

In [ ]:
%tensorboard --logdir logs/fit

## __Load Data__

In [ ]:
data_songs = pd.read_csv('our data path')

#shuffle our data rows
data_songs_shuffled = data_songs.sample(frac = 1, random_state=RSEED).reset_index(drop = True)

## __Do train & test split__

In [ ]:
def train_test_split(df,percentage):#percentage as 0. we want to have for our train set
    trainset = df.sample(frac = percentage)
    testset = df.drop(trainset.index)
    return trainset,testset
    

Do this again and split the train set, so we can produce a validation set

In [ ]:
def train_validation_split(df,percentage):#percentage as 0.anything, we want to have for our train set
    trainset = df.sample(frac = percentage)
    evalset = df.drop(trainset.index)
    return trainset,evalset

We have to convert the test and train data into an array, which is the acceptable form of tensorflow and keras.

In [ ]:
train_data = np.array(trainset, dtype = 'float32')
test_data = np.array(testset, dtype = 'float32')
validation_data = np.array(valset, dtype = 'float32')


In [ ]:
'''#if we can use the normal way we can split with sklearn
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)'''

Reshape

## __Define our target__

In [ ]:
#our target is are the labels in the dataframe. For the cnn we don't have to split the data into X and y we can feed it only a part of
#the data as a train set and another part as a data set.

In [ ]:
#here we define our target by splitting the 
x_train = train_data[:,1:]/255

y_train = train_data[:,0]

x_test= test_data[:,1:]/255

y_test=test_data[:,0]

## __Define functions__

In [ ]:
#defining softmax function

def softmax_by_string(t):
        sh = K.shape(t)
        string_sm = []
        for i in range(N_STRINGS):
            string_sm.append(K.expand_dims(K.softmax(t[:,i,:]), axis=1))
        return K.concatenate(string_sm, axis=1)


## Building our CNN Model

In [13]:
def cnn_swizzle_model():       
        swizzle_model = tf.keras.swizzle_models.Sequential()
        swizzle_model.add(tf.keras.layers.InputLayer(input_shape=[FRAME_HEIGHT, FRAME_WIDTH, 1], name='image'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3),activation='relu'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        swizzle_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        swizzle_model.add(tf.keras.layers.Dropout(0.25))   
        swizzle_model.add(tf.keras.layers.Flatten())
        swizzle_model.add(tf.keras.layers.Dense(128, activation='relu'))
        swizzle_model.add(tf.keras.layers.Dropout(0.5))
        swizzle_model.add(tf.keras.layers.Dense(126, activation='relu'))
        swizzle_model.add(tf.keras.layers.Dense(N_CLASSES * N_STRINGS)) # no activation
        swizzle_model.add(tf.keras.layers.Reshape((N_CLASSES, N_STRINGS)))
        swizzle_model.add(tf.keras.layers.Activation(softmax_by_string))
        return swizzle_model

Define model metrics for the cnn

In [ ]:
swizzle_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001),metrics =['accuracy'])

## __Train CNN__

In [14]:
history = swizzle_model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    validation_data=(x_validate,y_validate),
)

784

In [ ]:
#show plots for our loss function and the accurancy
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training - Loss Function')

plt.subplot(2, 2, 2)
plt.plot(history.history['acc'], label='Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.legend()
plt.title('Train - Accuracy')

In [ ]:
#print results of our swizzle model metrics
score = swizzle_model.evaluate(x_test,y_test,verbose=0)
print('Test Loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))